In [80]:
import pandas as pd
import numpy as np
import dataImport as dataI
import random

In [27]:
train = pd.read_csv('../Data/youtube.csv')
# n_columns = len(prueba.columns)
# print columns_cant

# data_dict ={}

# for column in prueba.columns:
#     data_dict[column] = prueba.ix[:, column].tolist()
    
# print data_dict['windy']

train.head(20)

,Outlook,temps,humedity,windy,hours
0,1,3,1,0,25
1,1,3,1,1,30
2,2,3,1,0,46
3,3,2,1,0,45
4,3,1,0,0,52
5,3,1,0,1,23
6,2,1,0,1,43
7,1,2,1,0,35
8,1,1,0,0,38
9,3,2,0,0,46


In [91]:
train.temps.unique()
print train.loc[train['temps']==3,'hours'].values.flatten()
print  train.loc[train['temps']==3,'hours'].count()
print train['temps'].count()

columns = train.columns.tolist()
columns.remove('Outlook')
print columns

print 

[25 30 46 44]
4
14
['temps', 'humedity', 'windy', 'hours']



In [90]:
def std_value(df,column,value,target):
    std = np.std(df.loc[train[column]==value,target].values.flatten())
    len = df[column].count()
    len_value = df.loc[train[column]==value,target].count()
    prob = float(len_value)/len
    return std*prob

In [89]:
def calculate_std(df,column,target):
    stds = 0
    values_uniques = df[column].unique()
    for value in values_uniques:
        stds +=  std_value(df,column,value,target)
    return stds

In [88]:
def calculate_stds(df,columns,target):
    stds=[]
    for column in columns:
        stds.append((column,calculate_std(df,column,target)))
    return stds
        

In [87]:
def final_stds(stds_columns,target_std):
    stds=[]
    for tupple in stds_columns:
        stds.append((tupple[0],target_std - tupple[1]))
    return stds

In [86]:
def get_nrandom_columns(df,target,n_columns):
    tot_columns = df.columns.tolist()
    tot_columns.remove(target)
    return [ tot_columns[i] for i in sorted(random.sample(xrange(len(tot_columns)), n_columns)) ]

In [92]:
def get_split(df,target,n_columns):
    # Calculo de la std de la variable a predecir
    target_std = np.std(df.ix[:, target].tolist())
    
    columns = get_nrandom_columns(df,target,n_columns)
    
    # calculamos la desviacion standard de cada columna:
    # Esto es: la sumatoria de (la probabilidad de cada uno de los distintos valores
    # de una columna * la desviacion standar del mismo) 
    stds_columns = calculate_stds(df,columns,target)
    
    #restamos cada std de columna al std del target
    stds_columns = final_stds(stds_columns,target_std)
    
    
    print stds_columns
    
    max_std = max(stds_columns,key=lambda item:item[1])
    
    #devolvemos el nombre de la columna con mayor diferencia de std
    return max_std[0]

In [99]:
def build_tree(df,target,n_columns):
    # target = nombre de la columna a predecir
    # n_columns =  cantidad de columnas random a considerar en cada split
    # n_columns < len(df.columns)!!!
    # para bagging n_columns = len(df.columns) - 1
    print get_split(df,target,n_columns)
        
build_tree(train,'hours',4)

ValueError: sample larger than population